<a href="https://colab.research.google.com/github/drjhero/data294/blob/main/mckinneyChapter7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Cleaning and Preparation
Some analysts say cleaning and rearranging takes up [80% of their time](https://www.pragmaticinstitute.com/resources/articles/data/overcoming-the-80-20-rule-in-data-science/#:~:text=Data%20practitioners%20spend%2080%25%20of,known%20as%20the%20Pareto%20principle.)!

Pandas does a lot of stuff for us under the hood. Did you know that the descriptive statistics we have computed in this class skip missing values?

## Handling Missing Data

In [83]:
# Recall that Pandas uses not a number (NaN) to represent missing data. This is
# called a "sentinel" value by some folks.
import pandas as pd
import numpy as np

floatData = pd.Series([1.2, -3.5, np.nan, 0])
floatData

0    1.2
1   -3.5
2    NaN
3    0.0
dtype: float64

In [84]:
floatData.isna()

0    False
1    False
2     True
3    False
dtype: bool

It is a smart idea to investigate missing data to see if there are problems you need to address or biases that may be introduced.

In [85]:
# Note that None is treated as NaN
stringData = pd.Series(["aardvark", np.nan, None, "avocado"])
stringData

0    aardvark
1         NaN
2        None
3     avocado
dtype: object

In [86]:
stringData.isna()

0    False
1     True
2     True
3    False
dtype: bool

In [87]:
floatData = pd.Series([1, 2, None], dtype='float64')
floatData

0    1.0
1    2.0
2    NaN
dtype: float64

In [88]:
floatData.isna()

0    False
1    False
2     True
dtype: bool

We are spending time on this because we will use isna() frequently when trying to find missing data in practice. isna() tries to give a consistent way to do this across many different data types.

### Filtering Out Missing Data

In [89]:
data = pd.Series([1, np.nan, 3.5, np.nan, 7])
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

In [90]:
# Here is an ugly way of doing the same thing
data[data.notna()]

0    1.0
2    3.5
4    7.0
dtype: float64

Pandas provides methods for picking how to drop missing elements from DataFrames.

In [91]:
data = pd.DataFrame([
    [1., 6.5, 3.],
    [1., np.nan, np.nan],
    [np.nan, np.nan, np.nan],
    [np.nan, 6.5, 3.]
])
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [92]:
# Note the default behavior to drop any row with a NaN
data.dropna()

,0,1,2
0,1.0,6.5,3.0


In [93]:
data.dropna(how="all")

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


In [94]:
data[4] = np.nan
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [95]:
data.dropna(axis="columns", how="all")

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [96]:
# We can set number of allowed missing observations
dataFrame = pd.DataFrame(np.random.standard_normal((7, 3)))
dataFrame

,0,1,2
0,1.007189,-1.296221,0.274992
1,0.228913,1.352917,0.886429
2,-2.001637,-0.371843,1.669025
3,-0.438570,-0.539741,0.476985
4,3.248944,-1.021228,-0.577087
5,0.124121,0.302614,0.523772
6,0.000940,1.343810,-0.713544


In [97]:
dataFrame.iloc[:4, 1] = np.nan
dataFrame.iloc[:2, 2] = np.nan
dataFrame

,0,1,2
0,1.007189,NaN,NaN
1,0.228913,NaN,NaN
2,-2.001637,NaN,1.669025
3,-0.438570,NaN,0.476985
4,3.248944,-1.021228,-0.577087
5,0.124121,0.302614,0.523772
6,0.000940,1.343810,-0.713544


In [98]:
dataFrame.dropna()

,0,1,2
4,3.248944,-1.021228,-0.577087
5,0.124121,0.302614,0.523772
6,0.000940,1.343810,-0.713544


In [99]:
dataFrame.dropna(thresh=2) # Rows with 2 or more NaNs

,0,1,2
2,-2.001637,NaN,1.669025
3,-0.438570,NaN,0.476985
4,3.248944,-1.021228,-0.577087
5,0.124121,0.302614,0.523772
6,0.000940,1.343810,-0.713544


### Filling in Missing Data

In [100]:
# fillna lets us replace NaNs with a simple value
# Using averages or median values is often better than zero in practice
dataFrame.fillna(0)

,0,1,2
0,1.007189,0.000000,0.000000
1,0.228913,0.000000,0.000000
2,-2.001637,0.000000,1.669025
3,-0.438570,0.000000,0.476985
4,3.248944,-1.021228,-0.577087
5,0.124121,0.302614,0.523772
6,0.000940,1.343810,-0.713544


In [101]:
# If you know specific values for specific missing values in each column we can
# supply a dictionary
dataFrame.fillna({1: 0.5, 2: 0})

,0,1,2
0,1.007189,0.500000,0.000000
1,0.228913,0.500000,0.000000
2,-2.001637,0.500000,1.669025
3,-0.438570,0.500000,0.476985
4,3.248944,-1.021228,-0.577087
5,0.124121,0.302614,0.523772
6,0.000940,1.343810,-0.713544


In [102]:
# We can specify simple interpolation methods
dataFrame = pd.DataFrame(np.random.standard_normal((6, 3)))
dataFrame

,0,1,2
0,-0.831154,-2.370232,-1.860761
1,-0.860757,0.560145,-1.265934
2,0.119827,-1.063512,0.332883
3,-2.359419,-0.199543,-1.541996
4,-0.970736,-1.307030,0.286350
5,0.377984,-0.753887,0.331286


In [103]:
dataFrame.iloc[2:, 1] = np.nan
dataFrame.iloc[4:, 2] = np.nan
dataFrame

,0,1,2
0,-0.831154,-2.370232,-1.860761
1,-0.860757,0.560145,-1.265934
2,0.119827,NaN,0.332883
3,-2.359419,NaN,-1.541996
4,-0.970736,NaN,NaN
5,0.377984,NaN,NaN


In [104]:
dataFrame.fillna(method="ffill") # Repeats the last number

,0,1,2
0,-0.831154,-2.370232,-1.860761
1,-0.860757,0.560145,-1.265934
2,0.119827,0.560145,0.332883
3,-2.359419,0.560145,-1.541996
4,-0.970736,0.560145,-1.541996
5,0.377984,0.560145,-1.541996


In [105]:
dataFrame.fillna(method="ffill", limit=2)

,0,1,2
0,-0.831154,-2.370232,-1.860761
1,-0.860757,0.560145,-1.265934
2,0.119827,0.560145,0.332883
3,-2.359419,0.560145,-1.541996
4,-0.970736,NaN,-1.541996
5,0.377984,NaN,-1.541996


In [106]:
# Lastly you can use statistics approaches which often are great
data = pd.Series([1., np.nan, 3.5, np.nan, 7])
data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [107]:
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

## Data Transformation

### Removing Duplicates

In [108]:
data = pd.DataFrame({
    "k1": ["one", "two"] * 3 + ["two"],
    "k2": [1, 1, 2, 3, 3, 4, 4]
})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [109]:
# We can easily check if any rows show up more than once.
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [110]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [111]:
data["v1"] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [112]:
# We can specify which parts of the data to check duplicates for
data.drop_duplicates(subset=["k1"])

,k1,k2,v1
0,one,1,0
1,two,1,1


### Transforming Data Using a Function or Mapping

In [113]:
meatData = pd.DataFrame({
    "food": ["bacon", "pulled pork", "bacon", "pastrami", "corned beef", "bacon", "pastrami", "honey ham", "nova lox"],
    "ounces": [4, 3, 12, 6, 7.5, 8, 3, 5, 6]
})
meatData

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,pastrami,6.0
4,corned beef,7.5
5,bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [114]:
# Next we are giving a mapping of each food to the type of meat in it
meatToAnimal = {
    "bacon": "pig",
    "pulled pork": "pig",
    "pastrami": "cow",
    "corned beef": "cow",
    "honey ham": "pig",
    "nova lox": "salmon"
}
meatData["animal"] = meatData["food"].map(meatToAnimal) # This does a ton with very little code
meatData

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,pastrami,6.0,cow
4,corned beef,7.5,cow
5,bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [115]:
# We can also use a custom function. This is less elegant here, but if
# the transformation doesn't work as a simple map this gives a lot of flexibility.
def getAnimal(x):
  return meatToAnimal[x]

meatData["food"].map(getAnimal)

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

### Replacing Values

In [116]:
data = pd.Series([1., -999., 2., -999., -2., 3.])
data

0      1.0
1   -999.0
2      2.0
3   -999.0
4     -2.0
5      3.0
dtype: float64

In [117]:
# Maybe those -999 are sentinels that mean missing
data.replace(-999, np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4   -2.0
5    3.0
dtype: float64

In [118]:
data.replace([-999, -2], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [119]:
data.replace([-999, -2], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [120]:
# If it is complicated we can throw a whole dictionary at it.
data.replace({-999: np.nan, -2: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### Renaming Axis Indexes

In [121]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=["Ohio", "Colorado", "New York"],
                    columns=["one", "two", "three", "four"])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [122]:
def transform(x):
  return x[:4].upper()
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [123]:
# If we want a transformed version of the original without modifying it
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [124]:
# As usual, we can specify a map for full control
data.rename(index={"Ohio": "INDIANA"},
            columns={"three": "peekaboo"})

,one,two,peekaboo,four
INDIANA,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


### Discretization and binning

In [125]:
# Given a list of ages, let's put them into "buckets" of
# [15, 25], [26, 35], [36, 60], [61, ...]
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]
bins = [18, 25, 35, 60, 100]
ageBuckets = pd.cut(ages, bins)
ageBuckets
# The output is Ca Categorical object

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64, right]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [126]:
ageBuckets.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [127]:
ageBuckets.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]], dtype='interval[int64, right]')

In [128]:
ageBuckets.categories[0]

Interval(18, 25, closed='right')

In [129]:
pd.value_counts(ageBuckets)

(18, 25]     5
(25, 35]     3
(35, 60]     3
(60, 100]    1
Name: count, dtype: int64

In [130]:
# Note the default to inclusive on the right which we can change
pd.cut(ages, bins, right=False)

[[18, 25), [18, 25), [25, 35), [25, 35), [18, 25), ..., [25, 35), [60, 100), [35, 60), [35, 60), [25, 35)]
Length: 12
Categories (4, interval[int64, left]): [[18, 25) < [25, 35) < [35, 60) < [60, 100)]

In [131]:
# We can add nice names to the Categories we build
groupNames = ["Youth", "YoungAdult", "MiddleAged", "Senior"]
pd.cut(ages, bins, labels=groupNames)

['Youth', 'Youth', 'Youth', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

In [132]:
# Pandas can compute equal length bins if we just give a number of bins instead
# of an ad hoc bin definition.
data = np.random.uniform(size=20)
pd.cut(data, 4, precision =2) # Limit decimal precision

[(0.79, 0.97], (0.6, 0.79], (0.79, 0.97], (0.41, 0.6], (0.22, 0.41], ..., (0.6, 0.79], (0.22, 0.41], (0.41, 0.6], (0.22, 0.41], (0.22, 0.41]]
Length: 20
Categories (4, interval[float64, right]): [(0.22, 0.41] < (0.41, 0.6] < (0.6, 0.79] < (0.79, 0.97]]

In [133]:
# We can also use sample quantiles
data = np.random.standard_normal(1000)
quartiles = pd.qcut(data, 4, precision = 2)
quartiles

[(0.62, 3.93], (-2.96, -0.68], (0.62, 3.93], (0.62, 3.93], (0.62, 3.93], ..., (-2.96, -0.68], (0.62, 3.93], (-0.026, 0.62], (-0.026, 0.62], (-2.96, -0.68]]
Length: 1000
Categories (4, interval[float64, right]): [(-2.96, -0.68] < (-0.68, -0.026] < (-0.026, 0.62] <
                                           (0.62, 3.93]]

In [134]:
pd.value_counts(quartiles)

(-2.96, -0.68]     250
(-0.68, -0.026]    250
(-0.026, 0.62]     250
(0.62, 3.93]       250
Name: count, dtype: int64

In [135]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.]).value_counts()

(-2.9499999999999997, -1.187]    100
(-1.187, -0.0265]                400
(-0.0265, 1.287]                 400
(1.287, 3.928]                   100
Name: count, dtype: int64

### Detecting and Filtering Outliers
There aren't a ton of fancy built in functions for this, its just basic array operations.

In [136]:
data = pd.DataFrame(np.random.standard_normal((1000, 4)))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.045383,0.024079,-0.004794,-0.050389
std,0.997796,1.008737,0.996599,0.996706
min,-3.645860,-3.184377,-3.745356,-3.428254
25%,-0.600254,-0.612162,-0.697084,-0.747478
50%,0.040150,-0.013609,-0.029924,-0.088274
75%,0.746527,0.690847,0.699046,0.623331
max,2.653656,3.525865,2.735527,3.366626


In [137]:
# Assume we wanted to find values larger than 3
col = data[2]
col[col.abs() > 3]

38    -3.399312
133   -3.745356
Name: 2, dtype: float64

In [138]:
# To "select" these rows we can use the any() method
data[(data.abs() > 3).any(axis="columns")]

,0,1,2,3
38,0.457246,-0.025907,-3.399312,-0.974657
57,1.951312,3.260383,0.963301,1.201206
133,0.508391,-0.196713,-3.745356,-1.520113
232,-0.242459,-3.056990,1.918403,-0.578828
255,0.682841,0.326045,0.425384,-3.428254
319,1.179227,-3.184377,1.369891,-1.074833
541,-3.548824,1.553205,-2.186301,1.277104
632,-0.578093,0.193299,1.397822,3.366626
779,-0.207434,3.525865,0.283070,0.544635
800,-3.645860,0.255475,-0.549574,-1.907459


In [139]:
# One way to handle outliers is to cap them at a maximum value
data[data.abs() > 3] = np.sign(data) * 3 # This is a lot in one line
data.describe() # Note the new max values

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.046577,0.023534,-0.003650,-0.050328
std,0.993777,1.005496,0.992788,0.994152
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.600254,-0.612162,-0.697084,-0.747478
50%,0.040150,-0.013609,-0.029924,-0.088274
75%,0.746527,0.690847,0.699046,0.623331
max,2.653656,3.000000,2.735527,3.000000


### Permutation and Random Sampling

In [140]:
# Permuting is th eprocess of randomly reordering a Series or the rows in a DataFrame

In [141]:
dataFrame = pd.DataFrame(np.arange(5 * 7).reshape((5, 7)))
dataFrame

,0,1,2,3,4,5,6
0,0,1,2,3,4,5,6
1,7,8,9,10,11,12,13
2,14,15,16,17,18,19,20
3,21,22,23,24,25,26,27
4,28,29,30,31,32,33,34


In [142]:
sampler = np.random.permutation(5)
sampler # This is just an array of the indexes of the rows in a random order

array([2, 4, 1, 0, 3])

In [143]:
dataFrame.take(sampler) # "takes" a given set of rows

,0,1,2,3,4,5,6
2,14,15,16,17,18,19,20
4,28,29,30,31,32,33,34
1,7,8,9,10,11,12,13
0,0,1,2,3,4,5,6
3,21,22,23,24,25,26,27


In [144]:
# Or if you really like iloc()
dataFrame.iloc[sampler]

,0,1,2,3,4,5,6
2,14,15,16,17,18,19,20
4,28,29,30,31,32,33,34
1,7,8,9,10,11,12,13
0,0,1,2,3,4,5,6
3,21,22,23,24,25,26,27


In [145]:
# We can also permute (new word alert) the columns
columnSampler = np.random.permutation(7)
columnSampler

array([0, 5, 2, 1, 6, 4, 3])

In [146]:
dataFrame.take(columnSampler, axis="columns")

,0,5,2,1,6,4,3
0,0,5,2,1,6,4,3
1,7,12,9,8,13,11,10
2,14,19,16,15,20,18,17
3,21,26,23,22,27,25,24
4,28,33,30,29,34,32,31


In [147]:
# This sets us up for sampling
# What if we want to take a random sample without replacement?
dataFrame.sample(n=3)

,0,1,2,3,4,5,6
0,0,1,2,3,4,5,6
3,21,22,23,24,25,26,27
2,14,15,16,17,18,19,20


In [148]:
# What if we want to take a random sample with replacement?
choices = pd.Series([5, 7, -1, 6, 4])
choices.sample(n=10, replace=True)

4    4
1    7
4    4
0    5
3    6
0    5
1    7
0    5
4    4
3    6
dtype: int64

### Computing Indicator Variables
Often times in modeling and machine learning problems, we want to convert
categorical variables into "indicator variables".
In Deep Learning books the like to call this one-hot encoding. If we have a
categorical variable with k possible values, we want to convert that into k
boolean columns.

In [149]:
dataFrame = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "b"],
                          "data1": range(6)})
dataFrame

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [150]:
pd.get_dummies(dataFrame["key"])

,a,b,c
0,False,True,False
1,False,True,False
2,True,False,False
3,False,False,True
4,True,False,False
5,False,True,False


In [151]:
# We can add prefixes to the column names easily
dummies = pd.get_dummies(dataFrame["key"], prefix="key")
dataFrameWithDummy = dataFrame[["data1"]].join(dummies)
dataFrameWithDummy

,data1,key_a,key_b,key_c
0,0,False,True,False
1,1,False,True,False
2,2,True,False,False
3,3,False,False,True
4,4,True,False,False
5,5,False,True,False


In [152]:
# Things get more complicated when we want to one-hot with multiple values per row
from google.colab import drive
drive.mount('/content/drive')
movieNames = ["movie_id", "title", "genres"]
movieData = pd.read_table('/content/drive/MyDrive/Teaching/DATA 294/data/movies.dat',
                          sep="::",
                          header=None,
                          names=movieNames,
                          engine="python")
movieData[:10]

Mounted at /content/drive


,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [153]:
# Note that genres is just a pipe-delimited ("|" is called pipe) list of genres
dummies = movieData["genres"].str.get_dummies("|")
dummies.iloc[:10, :6]

,Action,Adventure,Animation,Children's,Comedy,Crime
0,0,0,1,1,1,0
1,0,1,0,1,0,0
2,0,0,0,0,1,0
3,0,0,0,0,1,0
4,0,0,0,0,1,0
5,1,0,0,0,0,1
6,0,0,0,0,1,0
7,0,1,0,1,0,0
8,1,0,0,0,0,0
9,1,1,0,0,0,0


In [154]:
# We can give a nice prefix again
moviesWindic = movieData.join(dummies.add_prefix("Genre_"))
moviesWindic.iloc[0]

movie_id                                       1
title                           Toy Story (1995)
genres               Animation|Children's|Comedy
Genre_Action                                   0
Genre_Adventure                                0
Genre_Animation                                1
Genre_Children's                               1
Genre_Comedy                                   1
Genre_Crime                                    0
Genre_Documentary                              0
Genre_Drama                                    0
Genre_Fantasy                                  0
Genre_Film-Noir                                0
Genre_Horror                                   0
Genre_Musical                                  0
Genre_Mystery                                  0
Genre_Romance                                  0
Genre_Sci-Fi                                   0
Genre_Thriller                                 0
Genre_War                                      0
Genre_Western       

In [155]:
# A cute trick is using get_dummies() with pandas.cut() from earlier
# This gives us an easy way to create "buckets" and then get a one-hot
# encoding based on those buckets.
np.random.seed(12345) # Makes the results the same
values = np.random.uniform(size=10)
values

array([0.92961609, 0.31637555, 0.18391881, 0.20456028, 0.56772503,
       0.5955447 , 0.96451452, 0.6531771 , 0.74890664, 0.65356987])

In [156]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,False,False,False,False,True
1,False,True,False,False,False
2,True,False,False,False,False
3,False,True,False,False,False
4,False,False,True,False,False
5,False,False,True,False,False
6,False,False,False,False,True
7,False,False,False,True,False
8,False,False,False,True,False
9,False,False,False,True,False


### String manipulation

In [157]:
# A lot of the data we get will have messy delimited lists
val = "a,b,     pikachu"
val.split(",")

['a', 'b', '     pikachu']

In [158]:
# If we want to clear out whitespace
pieces = [x.strip() for x in val.split(",")]
pieces

['a', 'b', 'pikachu']

In [159]:
# If we instead want to put strings together in an ugly way
first, second, third = pieces # I hate this syntax
first + "::" + second + "::" + third

'a::b::pikachu'

In [160]:
"::".join(pieces)

'a::b::pikachu'

In [161]:
# If we want to check for a string's presence, "in" is popular
"pikachu" in val

True

In [162]:
# But we could also do
val.index(",")

1

In [163]:
# Or
val.find(":")

-1

In [164]:
# Note the different behavior of index when the string is not present
val.index("nope")

ValueError: substring not found

In [ ]:
# Similar to find is count()
val.count(",")

In [ ]:
# Replacement strings are part of the core python library
val.replace(",", "::")

### Regular Expressions
These are powerful ways to search strings but they are unreadable. I use them infrequently enough that I have to relearn them every time.

In [ ]:
import re
text = "foo      bar\t baz   \tqux" # \t means "tab"
re.split(r"\s+", text) # \s+ means "blankspace"

In [ ]:
# Or we can get everything that matches the expression, i.e., get all the blanks
regex = re.compile(r"\s+") # This is faster if we are going to use it again
regex.findall(text)

In [ ]:
# Consider this messy example of extracting email addresses
text = """Dave dave@google.com Steve steve@gmail.com Rob rob@gmail.com Ryan ryan@yahoo.com"""
pattern = r"[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}"
regex = re.compile(pattern, flags=re.IGNORECASE)
regex.findall(text)

In [ ]:
m = regex.search(text)
m

In [ ]:
text[m.start():m.end()]

In [ ]:
# Match only checks if the string starts with the pattern
print(regex.match(text))

In [ ]:
# sub() replaces matches
print(regex.sub("REDACTED", text))

In [ ]:
# What if we want to get the constituent parts
pattern = r"([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})"
regex = re.compile(pattern, flags=re.IGNORECASE)
m = regex.match("wesm@bright.net")
m.groups()

In [ ]:
# findall() in this case gets all the groups
regex.findall(text)

### String Functions in pandas
Cleaning up messy data sets from scratch can take a lot of work, especially when we don't know all the sentinel values and when we have missing data.

In [ ]:
data = {
    "Dave": "dave@google.com",
    "Steve": "steve@gmail.com",
    "Rob": "rob@gmail.com",
    "Wes": np.nan
}
data = pd.Series(data)
data

In [ ]:
data.isna()

In [ ]:
# Note how python treats the NaN results
data.str.contains("gmail") # str just contains a bunch of string functions and extensions like "[]"

In [ ]:
dataAsString = data.astype('string')
dataAsString

In [ ]:
dataAsString.str.contains("gmail") # This behaves more as expected

In [ ]:
pattern = r"([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})"
data.str.findall(pattern, flags=re.IGNORECASE)

In [ ]:
# If we want to get the captured groups
matches = data.str.findall(pattern, flags=re.IGNORECASE).str[0]
matches

In [ ]:
matches.str.get(1)

In [ ]:
data.str[:5]

In [ ]:
# This method elegantly gets the captured groups in a DataFrame
data.str.extract(pattern, flags=re.IGNORECASE)

## Categorical Data
Often times, we will come across columns that comprise categories, i.e., small sets of distinct values. Examples could be, types of food or days of the week.



In [165]:
values = pd.Series(['apple', 'orange', 'apple', 'apple'] * 2)
values

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [166]:
# In this example, we are dealing with two categories of fruit
pd.unique(values)

array(['apple', 'orange'], dtype=object)

In [167]:
pd.value_counts(values)

apple     6
orange    2
Name: count, dtype: int64

In [168]:
# One way to save computation time is to replace these (arbitrarily long) strings
# with short integer IDs and then store a table to lookup the names
values = pd.Series([0, 1, 0, 0] * 2)
names = pd.Series(['apple', 'orange'])
values

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

In [170]:
names

0     apple
1    orange
dtype: object

In [171]:
# It is easy to recreate the full-named list with take()
names.take(values)
# This representation of categorical data can really help performance
# especially when we are working with large data sets

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

### Categorical Extension Type in pandas

In [174]:
# Categorical is a special data type in pandas for working with this kind of data
# Under the hood its using the encoding we discussed above
fruits = ['apple', 'orange', 'apple', 'apple'] * 2
length = len(fruits)
rng = np.random.default_rng(seed=12345)
dataFrame = pd.DataFrame({'fruit': fruits,
                          'basket_id': np.arange(length),
                          'count': rng.integers(3, 15, size=length),
                          'weight': rng.uniform(0, 4, size=length)},
                         columns=['basket_id', 'fruit', 'count', 'weight'])
dataFrame

,basket_id,fruit,count,weight
0,0,apple,11,1.564438
1,1,orange,5,1.331256
2,2,apple,12,2.393235
3,3,apple,6,0.746937
4,4,apple,5,2.691024
5,5,orange,12,3.767211
6,6,apple,10,0.992983
7,7,apple,11,3.795525


In [175]:
# The fruit column is an array of string objects, if we want to convert it to Categorical
fruitCategorical = dataFrame['fruit'].astype('category')
fruitCategorical

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

In [178]:
c = fruitCategorical.array
type(c)

pandas.core.arrays.categorical.Categorical

In [179]:
c.categories

Index(['apple', 'orange'], dtype='object')

In [180]:
c.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

In [181]:
# If we want to get dictionary of the code name mappings
dict(enumerate(c.categories))

{0: 'apple', 1: 'orange'}

In [182]:
# If we want to convert the DataFrame column (instead of creating a copy)
dataFrame['fruit'] = dataFrame['fruit'].astype('category')
dataFrame['fruit']

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

In [183]:
# We can get Categorical directly when making a sequenve
categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])
categories

['foo', 'bar', 'baz', 'foo', 'bar']
Categories (3, object): ['bar', 'baz', 'foo']

In [187]:
# If we read the categorical data encoded from, for example, a CSV we can use from_codes
categories = ['foo', 'bar', 'baz']
codes = [0, 1, 2, 0, 0, 1]
categorical = pd.Categorical.from_codes(codes, categories)
categorical

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo', 'bar', 'baz']

In [189]:
# Categories don't have an 'order' by default, but feel free to give it one if
# that excites you
categorical = pd.Categorical.from_codes(codes, categories, ordered=True)
categorical

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

### Computations with Categorical

In [190]:
# Let's create categories by binning random numbers like we did eariler
rng = np.random.default_rng(seed=12345)
draws = rng.standard_normal(1000)
draws[:5]

array([-1.42382504,  1.26372846, -0.87066174, -0.25917323, -0.07534331])

In [191]:
bins = pd.qcut(draws, 4)
bins # Note these do get an order by default, which makes sense

[(-3.121, -0.675], (0.687, 3.211], (-3.121, -0.675], (-0.675, 0.0134], (-0.675, 0.0134], ..., (0.0134, 0.687], (0.0134, 0.687], (-0.675, 0.0134], (0.0134, 0.687], (-0.675, 0.0134]]
Length: 1000
Categories (4, interval[float64, right]): [(-3.121, -0.675] < (-0.675, 0.0134] < (0.0134, 0.687] <
                                           (0.687, 3.211]]

In [193]:
# Let's give them nicer names though
bins = pd.qcut(draws, 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
bins

['Q1', 'Q4', 'Q1', 'Q2', 'Q2', ..., 'Q3', 'Q3', 'Q2', 'Q3', 'Q2']
Length: 1000
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

In [194]:
# Now the boundaries are gone
bins = pd.Series(bins, name='quartile')
results = (pd.Series(draws)
                .groupby(bins)
                .agg(['count', 'min', 'max'])
                .reset_index()) # We will explain these better in the next module
results

,quartile,count,min,max
0,Q1,250,-3.119609,-0.678494
1,Q2,250,-0.673305,0.008009
2,Q3,250,0.018753,0.686183
3,Q4,250,0.688282,3.211418


In [195]:
# Note that quartile is the column with the categories that we are interested in
results['quartile']

0    Q1
1    Q2
2    Q3
3    Q4
Name: quartile, dtype: category
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

In [197]:
# Earlier we asserted that Categorical computations were more performant, let's test it!
n = 10_000_000
labels = pd.Series(['foo', 'bar', 'baz', 'qux'] * (n // 4))
categories = labels.astype('category')
labels.memory_usage(deep=True)

600000128

In [198]:
categories.memory_usage(deep=True)

10000540

In [199]:
# The cost of converting it is a one time cost that isn't terrible in this example
%time _ = labels.astype('category')

CPU times: user 504 ms, sys: 222 ms, total: 726 ms
Wall time: 723 ms


In [200]:
%timeit labels.value_counts()

538 ms ± 12.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [201]:
# This is much faster on a O(n) function like counting values
%timeit categories.value_counts()

55.8 ms ± 6.39 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Categorical Methods

In [202]:
series = pd.Series(['a', 'b', 'c', 'd'] * 2)
seriesCategorical = series.astype('category')
seriesCategorical

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [203]:
# "cat" has several Categorical helper functions
seriesCategorical.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [204]:
seriesCategorical.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

In [205]:
# Imagine there are categories we know to exist but which do not appear in the data we read
actualCategories = ['a', 'b', 'c', 'd', 'e']
seriesCategorical2 = seriesCategorical.cat.set_categories(actualCategories)
seriesCategorical2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): ['a', 'b', 'c', 'd', 'e']

In [206]:
# Note how the raw data looks the same, but the helper functions will pick up on the missing category
seriesCategorical.value_counts()

a    2
b    2
c    2
d    2
Name: count, dtype: int64

In [207]:
seriesCategorical2.value_counts()

a    2
b    2
c    2
d    2
e    0
Name: count, dtype: int64